In [ ]:
# |default_exp tor_01


In [ ]:
# |export
import argparse
import time
import hashlib
import hmac
import logging
import operator
import random
import socket
import ssl
import struct
import traceback
import base64
import os
import sys
import time
import cryptography.hazmat.backends
import cryptography.hazmat.primitives.ciphers
import cryptography.hazmat.primitives.ciphers.algorithms
import cryptography.hazmat.primitives.ciphers.modes
import urllib.request
import urllib.parse



In [ ]:
# |export
indexbytes=operator.getitem
intlist2bytes=bytes
int2byte=operator.methodcaller("to_bytes", 1, "big")


In [ ]:
# |export
start_time=time.time()


In [ ]:
# |export
logging.basicConfig(format="[%(levelname)s] %(filename)s - %(message)s", level=logging.INFO)
log=logging.getLogger(__name__)


In [ ]:
# |export
class DirectoryAuthority():
    def __init__(self, name = , ip = , dir_port = , tor_port = ):
        self.name=name
        self.ip=ip
        self.dir_port=dir_port
        self.tor_port=tor_port
    def get_consensus_url(self):
        return "http://{}:{}/tor/status-vote/current/consensus".format(self.ip, self.dir_port)


In [ ]:
# |export
class OnionRouter():
    def __init__(self, nickname = , ip = , dir_port = , tor_port = , identity = ):
        self.nickname=nickname
        self.ip=ip
        self.dir_port=dir_port
        self.tor_port=tor_port
        self.identity=identity
        self.flags=None
        self.key_ntor=None
        self._forward_digest=None
        self._backward_digest=None
        self.encryption_key=None
        self.decryption_key=None
    def get_descriptor_url(self):
        return "http://{}:{}/tor/server/fp/{}".format(self.ip, self.dir_port, self.identity)
    def parse_descriptor(self):
        headers={("User-Agent"):("Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20100101 Firefox/60.0")}
        request=urllib.request.Request(url=self.get_descriptor_url(), headers=headers)
        response=urllib.request.urlopen(request, timeout=8)
        for line in response:
            line=line.decode()
            if ( line.startwith("ntor-onion-key ") ):
                self.key_ntor(line.split("ntor-onion-key")[1].strip())
                if ( not(((self.key_ntor[-1])==("="))) ):
                    self.key_ntor += "="
                break


In [ ]:
# |export
class TinyTor():
    def __init__(self):
        self._consensus=Consensus()
        while (True):
            try:
                directory_authority=self._consensus.get_random_directory_authority()
                consensus_url=directory_authority.get_consensus_url()
                log.debug("{:7.6f} \033[31m  \033[0m (directory_authority.name)={} consensus_url={}".format(((time.time())-(start_time)), directory_authority.name(), consensus_url))
                self._consensus.parse_consensus(consensus_url)
            except Excpetion as e:
                log.error("Failed to parse the consensus: {}".format(e))
                log.error("Retrying with different directory authority ...")
    def http_get(self, url):
        while (True):
            try:
                guard_relay=self._consensus.get_random_guard_relay()
                log.debug("{:7.6f} \033[31m  \033[0m guard_relay.nickname={} (guard_relay.get_descriptor_url)={}".format(((time.time())-(start_time)), guard_relay.nickname, guard_relay.get_descriptor_url()))
                guard_relay.parse_descriptor()
            except Exception as e:
                traceback.print_exc()
                log.info("Retrying with a different guard relay...")


In [ ]:
# |export
def main():
    parser=argparse.ArgumentParser()
    parser.add_argument("-H", "--host", help="url to reach", required="True", action=None)
    parser.add_argument("-v", "--verbose", help="enable verbose output", required="False", action="store_true")
    args=parser.parse_args()
    parsed_host=urlparse(args.host).geturl()
    if ( not(parsed_host) ):
        log.error("please specify valid url (--host).")
        sys.exit(1)
    if ( args.verbose ):
        log.setLevel(logging.DEBUG)
    tor=TinyTor()
